## Deliverable 2. Create a Customer Travel Destinations Map.

In [20]:
! pip install gmaps

In [21]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [22]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/bryonlhobbs/Desktop/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,2,Tefe,BR,77.11,light rain,-3.3542,-64.7114,Aracati Hotel
1,10,Rikitea,PF,79.30,few clouds,-23.1203,-134.9692,People ThankYou
2,11,Bathsheba,BB,81.21,broken clouds,13.2167,-59.5167,Atlantis Hotel
3,12,Hithadhoo,MV,82.92,broken clouds,-0.6000,73.0833,Scoop Guest House
4,13,Vaini,TO,81.34,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [23]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,2,Tefe,BR,77.11,light rain,-3.3542,-64.7114,Aracati Hotel
1,10,Rikitea,PF,79.30,few clouds,-23.1203,-134.9692,People ThankYou
2,11,Bathsheba,BB,81.21,broken clouds,13.2167,-59.5167,Atlantis Hotel
3,12,Hithadhoo,MV,82.92,broken clouds,-0.6000,73.0833,Scoop Guest House
4,13,Vaini,TO,81.34,few clouds,-21.2000,-175.2000,Keleti Beach Resort
5,17,Arraial Do Cabo,BR,76.86,clear sky,-22.9661,-42.0278,Pousada Porto Praia
6,21,Butaritari,KI,80.42,moderate rain,3.0707,172.7902,Isles Sunset Lodge
7,23,Auki,SB,80.38,scattered clouds,-8.7676,160.7034,Auki Motel
8,29,Hilo,US,77.11,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
9,32,Puerto Ayora,EC,84.16,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [25]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID        210
City           210
Country        210
Max Temp       210
Description    210
Lat            210
Lng            210
Hotel Name     210
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how="any")
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 0 to 209
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City_ID      195 non-null    int64  
 1   City         195 non-null    object 
 2   Country      195 non-null    object 
 3   Max Temp     195 non-null    float64
 4   Description  195 non-null    object 
 5   Lat          195 non-null    float64
 6   Lng          195 non-null    float64
 7   Hotel Name   195 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 13.7+ KB


In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Tefe,BR,77.11,light rain,-3.3542,-64.7114,
1,Rikitea,PF,79.30,few clouds,-23.1203,-134.9692,
2,Bathsheba,BB,81.21,broken clouds,13.2167,-59.5167,
3,Hithadhoo,MV,82.92,broken clouds,-0.6000,73.0833,
4,Vaini,TO,81.34,few clouds,-21.2000,-175.2000,
5,Arraial Do Cabo,BR,76.86,clear sky,-22.9661,-42.0278,
6,Butaritari,KI,80.42,moderate rain,3.0707,172.7902,
7,Auki,SB,80.38,scattered clouds,-8.7676,160.7034,
8,Hilo,US,77.11,overcast clouds,19.7297,-155.0900,
9,Puerto Ayora,EC,84.16,light rain,-0.7393,-90.3518,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except Exception as err:
        print(err, index)




In [29]:
hotel_df.tail(20)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
188,Guaymas,MX,87.89,clear sky,27.9333,-110.9000,"Holiday Inn Express Guaymas, an IHG Hotel"
189,Houston,US,87.19,clear sky,29.7633,-95.3633,"Hotel ICON, Autograph Collection"
190,Sola,VU,84.87,scattered clouds,-13.8833,167.5500,Leumerus Bungalows
191,El Jicaro,GT,80.13,overcast clouds,14.9167,-89.9000,Auto Hotel Refugio de Don Pepe
192,Mayumba,GA,76.35,broken clouds,-3.4320,10.6554,LIKWALE LODGE
194,Urbano Santos,BR,75.74,moderate rain,-3.2078,-43.4036,Pousada Santa Dulce
195,Galveston,US,85.08,clear sky,29.3669,-94.9669,Crystal Suites
196,Morgan City,US,75.00,clear sky,29.6994,-91.2068,Hampton Inn & Suites Morgan City
197,Touho,NC,77.83,scattered clouds,-20.7833,165.2333,Auberge Timay Evasion
198,Mbandaka,CD,75.69,overcast clouds,0.0487,18.2603,Guest House Epervier


In [30]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[(hotel_df["Hotel Name"] == ""), ["Hotel Name"]] = np.nan
clean_hotel_df=hotel_df.dropna(subset=['Hotel Name'])

In [31]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/bryonlhobbs/Desktop/World_Weather_Analysis/Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))